# Programmation par contrainte: évaluation pratique
## 5-SDBD + mastère VALDOM, INSA Toulouse, janvier 2023

## Première partie

On utilise tout au long de ce sujet deux ensembles d’agents H = {h1, . . . hn} et F = {f1, . . . fn}. Chaque agent
hi ∈ H exprime ses préférences de couplage avec un agent de F à travers une liste L(hi) contenant les éléments de F
sans duplication. Pour tout k ∈ [1, n − 1], hi préfère être couplé avec le kème agent dans L(hi) que le k + 1ème agent.
De la même façon, chaque agent fj ∈ F exprime ses préférences de couplage avec un agent de H à travers une liste
L(fj ).

Un tuple (h, f ) est appelé un couple si h ∈ H et f ∈ F . Soit M un ensemble de couples. Un agent a préfère un
agent b à sa situation dans M si a n’appartient à aucun couple dans M ou si a préfère b à son partenaire dans M . Un
couple (h, f ) ∈ M bloque M si h préfère f à sa situation dans M et f préfère h à sa situation dans M .
Un mariage stable est un ensemble de couples M tel que chaque agent est couplé avec un seul agent et M n’admet
aucun couple bloquant. On note qu’une instance de ce problème peut admettre plusieurs solutions.

In [229]:
from config import setup
setup()

from docplex.cp.model import *
from docplex.cp.config import get_default

import numpy as np
%matplotlib inline

1. Proposez un modèle de programmation par contrainte pour trouver un mariage stable. Le modèle peut inclure
des contraintes logiques (par exemple if_then). L’utilisation de ce genre de contraintes est accessible dans la
documentation du solveur. Utilisez la recherche en profondeur par défaut ((’DepthFirst’) sans préciser des
heuristiques de branchements. N’affichez pas les traces d’exécutions internes du solveur.

In [298]:
from random import shuffle

def generate_prefs(n):
    
    Lh = []
    Lf = []
    for i in range(n):
        h = list(set(list(range(n))))
        f = list(set(list(range(n))))
        shuffle(h)
        Lh.append(h)
        shuffle(f)
        Lf.append(f)
        
    return Lh, Lf

In [299]:
#Lf, Lh = generate_prefs(4)

In [300]:
#Données du problème

#Nombre d'agents dans chaque groupe
n = 4

#Matrice de préférences de H (liste de listes)
#La préférence est décroissante : plus un agent a un indice bas, plus la préférence est élevée
Lh_old = [[2,3,1,4], 
      [4,1,3,2],
      [2,4,1,3],
      [3,1,4,2]]
one = np.ones((4,4), dtype = int)
Lh = (Lh_old - one).tolist()

#Matrice de préférences de F
Lf_old = [[2,1,3,4],
      [3,4,1,2],
      [1,3,4,2],
      [2,1,3,4]]
Lf = (Lf_old - one).tolist()

print(Lh, "\n\n", Lf)

[[1, 2, 0, 3], [3, 0, 2, 1], [1, 3, 0, 2], [2, 0, 3, 1]] 

 [[1, 0, 2, 3], [2, 3, 0, 1], [0, 2, 3, 1], [1, 0, 2, 3]]


In [331]:
def print_prefs(Lh,Lf):
    print("##___Lh___##")
    for i in range(n):
        print(Lh[i])
    print("\n")
    
    print("##___Lf___##")
    for i in range(n):
        print(Lf[i])

In [332]:
print_prefs(Lh,Lf)

##___Lh___##
[1, 2, 0, 3]
[3, 0, 2, 1]
[1, 3, 0, 2]
[2, 0, 3, 1]


##___Lf___##
[1, 0, 2, 3]
[2, 3, 0, 1]
[0, 2, 3, 1]
[1, 0, 2, 3]


In [301]:
#Représentation des couples : l'agent h_i est en couple avec l'agent f_couple[i]
#Exemple : si couple = [1, 3, 4, 2] alors l'agent h2 est en couple avec f4 
#couples = mdl.integer_var_list(n,0,n-1,'c')

In [346]:
def print_sol(sol,n):
    if sol.solve_status == "Feasible":
        res = "L'ensemble M={"
        for i in range(n):
            for j in range(n):
                if sol.get_value(couples[i][j]) == 1:
                    res = res + "(h_" + str(i) + ",f_" + str(j) + "),"
        res = res[:-1] + "}" + " est un mariage stable."

        print(res)
    else:
        print("Ce modèle ne possède pas de solutions.")

In [333]:
mdl = CpoModel(name='couples stables')
n=4

#Variables
couples = [mdl.binary_var_list(n, f"c{i}") for i in range(n)]
h_index = mdl.integer_var_list(n, 0, n-1, name='ih') 
f_index = mdl.integer_var_list(n, 0, n-1, name='if')


for i in range(n):
    #indice de fi dans Lh
    mdl.add(f_index[i] == mdl.sum([couples[i][j] * (Lh[i].index(j)) for j in range(n)]))
    #indice de hi dans Lf
    mdl.add(h_index[i] == mdl.sum([couples[j][i] * (Lf[i].index(j)) for j in range(n)]))
        
#Chaque agent ne peut être que dans un seul couple
for i in range(n):
    mdl.add(mdl.sum([couples[i][j] for j in range(n)]) == 1)
    mdl.add(mdl.sum([couples[j][i] for j in range(n)]) == 1)

# Blocking couple
for i in range(n):
    for j in range(n):
        mdl.add(mdl.if_then(Lh[i].index(j) < f_index[i], Lf[j].index(i)>=h_index[i]))

#print(mdl.get_cpo_string())

In [341]:
sol = mdl.solve(trace_log=False)

In [347]:
#sol.print_solution()
print_sol(sol,n)

L'ensemble M={(h_0,f_3),(h_1,f_0),(h_2,f_1),(h_3,f_2)} est un mariage stable.


## Seconde partie